# Sudoku Solver - OR-Tools et Z3 (Python)

Ce notebook implémente des solveurs de Sudoku utilisant:
- **Google OR-Tools CP-SAT**: Constraint Programming avec solveur SAT
- **Microsoft Z3**: SMT (Satisfiability Modulo Theories) solver

Ces approches sont équivalentes aux notebooks C# `Sudoku-3-ORTools.ipynb` et `Sudoku-4-Z3.ipynb`.

## Installation

```bash
pip install ortools z3-solver
```

In [ ]:
# Imports
import time
from typing import List, Optional, Tuple

# Vérifier les installations
try:
    from ortools.sat.python import cp_model
    print(f"OR-Tools importé avec succès")
except ImportError:
    print("OR-Tools non installé. Exécutez: pip install ortools")

try:
    from z3 import *
    print(f"Z3 importé avec succès")
except ImportError:
    print("Z3 non installé. Exécutez: pip install z3-solver")

## 1. Classe SudokuGrid (réutilisée)

In [ ]:
class SudokuGrid:
    """Représentation d'une grille de Sudoku 9x9."""
    
    def __init__(self, grid: Optional[List[List[int]]] = None):
        if grid is None:
            self.cells = [[0] * 9 for _ in range(9)]
        else:
            self.cells = [row[:] for row in grid]
    
    @classmethod
    def from_string(cls, s: str) -> 'SudokuGrid':
        s = s.replace('.', '0').replace(' ', '').replace('\n', '')
        if len(s) != 81:
            raise ValueError(f"La chaîne doit avoir 81 caractères")
        grid = cls()
        for i in range(81):
            grid.cells[i // 9][i % 9] = int(s[i])
        return grid
    
    def clone(self) -> 'SudokuGrid':
        return SudokuGrid(self.cells)
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.cells[r][c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger puzzles
easy_puzzles = load_puzzles('Puzzles/Sudoku_Easy51.txt', max_puzzles=10)
hard_puzzles = load_puzzles('Puzzles/Sudoku_hardest.txt')
print(f"Puzzles chargés: {len(easy_puzzles)} faciles, {len(hard_puzzles)} difficiles")

## 2. Solveur OR-Tools CP-SAT

Google OR-Tools utilise un solveur CP-SAT (Constraint Programming with SAT) très performant.

In [ ]:
class ORToolsSolver:
    """Solveur Sudoku utilisant OR-Tools CP-SAT."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout le Sudoku avec OR-Tools CP-SAT."""
        model = cp_model.CpModel()
        
        # Créer les variables: cells[i][j] in {1..9}
        cells = {}
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    # Valeur fixée
                    cells[(i, j)] = model.NewConstant(grid.cells[i][j])
                else:
                    # Variable libre
                    cells[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
        
        # Contraintes: toutes différentes par ligne
        for i in range(9):
            model.AddAllDifferent([cells[(i, j)] for j in range(9)])
        
        # Contraintes: toutes différentes par colonne
        for j in range(9):
            model.AddAllDifferent([cells[(i, j)] for i in range(9)])
        
        # Contraintes: toutes différentes par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                box_cells = []
                for i in range(3):
                    for j in range(3):
                        box_cells.append(cells[(box_row * 3 + i, box_col * 3 + j)])
                model.AddAllDifferent(box_cells)
        
        # Résoudre
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # Extraire la solution
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = solver.Value(cells[(i, j)])
            return True
        
        return False

# Test
ortools_solver = ORToolsSolver()
test_grid = SudokuGrid.from_string(hard_puzzles[0])
print("Puzzle difficile:")
print(test_grid)

start = time.time()
solved = ortools_solver.solve(test_grid)
elapsed = (time.time() - start) * 1000

print(f"\nRésolu: {solved} en {elapsed:.2f} ms")
print("\nSolution:")
print(test_grid)

## 3. Solveur Z3 SMT

Microsoft Z3 est un solveur SMT (Satisfiability Modulo Theories) très puissant.

In [ ]:
class Z3Solver:
    """Solveur Sudoku utilisant Z3 SMT."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout le Sudoku avec Z3."""
        # Créer les variables: X[i][j] est un entier 1-9
        X = [[Int(f'x_{i}_{j}') for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine: 1 <= X[i][j] <= 9
        for i in range(9):
            for j in range(9):
                solver.add(X[i][j] >= 1, X[i][j] <= 9)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes: toutes différentes par ligne
        for i in range(9):
            solver.add(Distinct([X[i][j] for j in range(9)]))
        
        # Contraintes: toutes différentes par colonne
        for j in range(9):
            solver.add(Distinct([X[i][j] for i in range(9)]))
        
        # Contraintes: toutes différentes par bloc 3x3
        for box_row in range(3):
            for box_col in range(3):
                box_cells = []
                for i in range(3):
                    for j in range(3):
                        box_cells.append(X[box_row * 3 + i][box_col * 3 + j])
                solver.add(Distinct(box_cells))
        
        # Résoudre
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Test
z3_solver = Z3Solver()
test_grid = SudokuGrid.from_string(hard_puzzles[0])
print("Puzzle difficile:")
print(test_grid)

start = time.time()
solved = z3_solver.solve(test_grid)
elapsed = (time.time() - start) * 1000

print(f"\nRésolu: {solved} en {elapsed:.2f} ms")
print("\nSolution:")
print(test_grid)

## 4. Comparaison des performances

In [ ]:
def benchmark_solvers(puzzles: List[str], name: str):
    """Compare les performances des solveurs."""
    print(f"\n{'='*60}")
    print(f"Benchmark: {name} ({len(puzzles)} puzzles)")
    print('='*60)
    
    solvers = {
        'OR-Tools CP-SAT': ORToolsSolver(),
        'Z3 SMT': Z3Solver()
    }
    
    results = {name: {'time': 0, 'solved': 0} for name in solvers}
    
    for puzzle_str in puzzles:
        for solver_name, solver in solvers.items():
            grid = SudokuGrid.from_string(puzzle_str)
            
            start = time.time()
            solved = solver.solve(grid)
            elapsed = time.time() - start
            
            results[solver_name]['time'] += elapsed
            if solved:
                results[solver_name]['solved'] += 1
    
    # Afficher résultats
    print(f"\n{'Solveur':<20} {'Résolus':<10} {'Temps total':<15} {'Temps moyen':<15}")
    print('-'*60)
    for solver_name, data in results.items():
        avg_time = (data['time'] / len(puzzles)) * 1000
        total_time = data['time'] * 1000
        print(f"{solver_name:<20} {data['solved']}/{len(puzzles):<8} {total_time:>10.2f} ms   {avg_time:>10.2f} ms")
    
    return results

# Benchmark
benchmark_solvers(easy_puzzles, "Puzzles Faciles")
benchmark_solvers(hard_puzzles, "Puzzles Difficiles (Top 11)")

## 5. Z3 Optimisé avec BitVectors

Version optimisée utilisant des bit-vectors 4 bits (comme dans le notebook C#).

In [ ]:
class Z3BitVectorSolver:
    """Solveur Z3 utilisant des BitVectors 4 bits."""
    
    def solve(self, grid: SudokuGrid) -> bool:
        """Résout avec BitVectors."""
        # BitVectors 4 bits (valeurs 1-9 tiennent dans 4 bits)
        X = [[BitVec(f'x_{i}_{j}', 4) for j in range(9)] for i in range(9)]
        
        solver = Solver()
        
        # Contraintes de domaine
        for i in range(9):
            for j in range(9):
                solver.add(UGE(X[i][j], 1))  # >= 1 (unsigned)
                solver.add(ULE(X[i][j], 9))  # <= 9 (unsigned)
        
        # Valeurs initiales
        for i in range(9):
            for j in range(9):
                if grid.cells[i][j] != 0:
                    solver.add(X[i][j] == grid.cells[i][j])
        
        # Contraintes Distinct (par paires pour BitVec)
        def add_all_different(cells):
            for i in range(len(cells)):
                for j in range(i + 1, len(cells)):
                    solver.add(cells[i] != cells[j])
        
        # Lignes
        for i in range(9):
            add_all_different([X[i][j] for j in range(9)])
        
        # Colonnes
        for j in range(9):
            add_all_different([X[i][j] for i in range(9)])
        
        # Blocs
        for box_row in range(3):
            for box_col in range(3):
                box_cells = [X[box_row * 3 + i][box_col * 3 + j] 
                             for i in range(3) for j in range(3)]
                add_all_different(box_cells)
        
        if solver.check() == sat:
            model = solver.model()
            for i in range(9):
                for j in range(9):
                    grid.cells[i][j] = model.evaluate(X[i][j]).as_long()
            return True
        
        return False

# Comparaison Z3 Int vs BitVec
print("=== Comparaison Z3 Int vs BitVector ===")

z3_int = Z3Solver()
z3_bv = Z3BitVectorSolver()

for i, puzzle_str in enumerate(hard_puzzles[:5]):
    print(f"\nPuzzle {i+1}:")
    
    grid1 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_int.solve(grid1)
    t1 = (time.time() - start) * 1000
    
    grid2 = SudokuGrid.from_string(puzzle_str)
    start = time.time()
    z3_bv.solve(grid2)
    t2 = (time.time() - start) * 1000
    
    print(f"  Z3 Int:      {t1:>8.2f} ms")
    print(f"  Z3 BitVec:   {t2:>8.2f} ms")

## 6. Résumé et conclusions

| Solveur | Avantages | Inconvénients |
|---------|-----------|---------------|
| **OR-Tools CP-SAT** | Très rapide, API simple | Moins flexible pour contraintes complexes |
| **Z3 Int** | Flexible, expressif | Plus lent que CP-SAT |
| **Z3 BitVec** | Optimisé pour petits domaines | Syntaxe plus complexe |

Pour le Sudoku, OR-Tools CP-SAT est généralement le plus rapide car il est optimisé pour les problèmes de satisfaction de contraintes.

## Navigation

- **Précédent**: [Sudoku-Python-Backtracking.ipynb](Sudoku-Python-Backtracking.ipynb) - Backtracking simple
- **Notebooks C# équivalents**: [Sudoku-3-ORTools.ipynb](Sudoku-3-ORTools.ipynb), [Sudoku-4-Z3.ipynb](Sudoku-4-Z3.ipynb)